# 0. Read Score

In [1]:
import xml.etree.cElementTree as ET

filename = "../../XMLs/至少還有你_主旋律.musicxml"
#filename = "../../XMLs/jinglebells.xml"
tree = ET.parse(filename)
root = tree.getroot()
root.tag, root.attrib

('score-partwise', {'version': '3.1'})

In [2]:
from mxlParsing import *

xml_measures = root.findall('.//measure')
measures = []

for xml_measure in xml_measures:
    measure = Measure(get_meas_notes_xml(xml_measure))
    if not measure.getSos()==['0']:
        measures.append(measure)
        
#make parts
m21Part_main = measures_to_m21Part(measures)

#get time signature
top,bottom = get_ts_xml(root)
ts = m21.meter.TimeSignature(str(top)+'/'+str(bottom))

#build
score = build_m21Score_1p(m21Part_main,'zhishaoyouni', ts)
score.show('lily.png')

In [ ]:
def xmlToScore(filename):
    tree = ET.parse(filename)
    root = tree.getroot()
    xml_measures = root.findall('.//measure')
    measures = []

    for xml_measure in xml_measures:
        measure = Measure(get_meas_notes_xml(xml_measure))
        if not measure.getSos()==['0']:
            measures.append(measure)
            
    return Score(measures)
    

# 1. Mess around

In [ ]:
import music21 as m21
key = score.analyze('key')
print(key.tonic.name, key.mode)

In [ ]:
# transpose
k = score.analyze('KrumhanslSchmuckler')
i = m21.interval.Interval(k.tonic, m21.pitch.Pitch('C'))
sNew = score.transpose(i)
sNew.show('lily.png')

In [ ]:
score.show('midi')

# 2. Getting co-occurence

In [ ]:
import pandas as pd

coocc_df = pd.read_pickle('co_occurance_dict.pickle')
coocc_df

In [ ]:
df = pd.DataFrame.from_dict(coocc_df)
df.head()

In [ ]:
df.index

In [ ]:
df = df.reset_index(level=[0,1])
df.head()

In [ ]:
df3.index

In [ ]:
df2 = df.groupby('level_0').sum()
df2

In [ ]:
df3 = df2.transpose()
df3

In [ ]:
df3 = df3.reset_index(level=[0,1])
df3

In [ ]:
df3 = df3.groupby('level_0').sum()
df3

In [ ]:
df3.to_pickle('note_coocc.pickle')

# 3. Fitness function

In [5]:
import pandas as pd

coocc_df = pd.read_pickle('note_coocc.pickle')
coocc_df

level_0,A#0,A#1,A#2,A#3,A#4,A#5,A#6,A#7,A0,A1,...,G6,G7,Gb0,Gb1,Gb2,Gb3,Gb4,Gb5,Gb6,Gb7
level_0,,,,,,,,,,,,,,,,,,,,,
A#0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A#1,0.0,10.0,99.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A#2,0.0,33.0,77.0,84.0,0.0,0.0,0.0,0.0,0.0,3.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
A#3,0.0,3.0,53.0,204.0,71.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0
A#4,0.0,1.0,9.0,92.0,427.0,137.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,15.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Gb3,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,2.0,...,3.0,0.0,0.0,8.0,119.0,306.0,154.0,11.0,0.0,0.0
Gb4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2.0,...,4.0,4.0,0.0,7.0,20.0,53.0,305.0,242.0,4.0,0.0
Gb5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,...,18.0,6.0,0.0,8.0,21.0,11.0,85.0,171.0,143.0,8.0


In [6]:
ml = measures[1].getSos()
ml

['F4', 'A4', 'D5', 'D5', 'D5', 'A4', 'A4']

In [7]:
import math

def get_measure_fitness(measure):
    '''get a fitness score for a measure'''
    m_score = 0
    rests = 0
    ml = measure.getSos()
    ml_len = len(ml)
    for index in range(0,ml_len-1):
        #handle naturals
        prev_note = ml[index]
        if len(ml[index])>2 and ml[index][1]=='n':
            ml[index] = ml[index][0]+ml[index][2]
        if len(ml[index+1])>2 and ml[index+1][1]=='n':
            ml[index+1] = ml[index+1][0]+ml[index+1][2]
        #can't compute rests
        if ml[index]=='0' or ml[index+1]=='0':
            rests += 1
        #get fitness
        else:
            note_score = coocc_df[ml[index]][ml[index+1]]
            m_score += note_score
    m_score /= (ml_len-rests)
    m_score = round(m_score,3)
    return m_score
            
        

In [8]:
def get_measarr_fitness(measures):
    score_fitness = 0
    for measure in measures:
        score_fitness+=get_measure_fitness(measure)
    score_fitness /= len(measures)
    score_fitness = round(score_fitness,2)
    return score_fitness

In [9]:
def get_score_fitness(score):
    measures = score.measures
    score_fitness = 0
    for measure in measures:
        score_fitness+=get_measure_fitness(measure)
    score_fitness /= len(measures)
    score_fitness = round(score_fitness,2)
    return score_fitness

In [10]:
for measure in measures:
    print(get_measure_fitness(measure))

3061.5
3078.143
3843.143
2747.0
3525.875
2799.875
2226.143
1580.167
3061.5
3078.143
3843.143
2941.889
3967.625
2799.875
2226.143
1943.0
2820.667
2024.182
2780.333
2686.455
1959.333
2024.182
2780.333
3231.182
0.0


In [11]:
get_score_fitness(Score(measures))

2681.19

# 4. Compare measures

In [12]:
def compare_measures(m1,m2):
    measure_diff = 0
    rests = 0
    m1_notes = m1.notes
    m2_notes = m2.notes
    min_len = min(len(m1_notes),len(m2_notes))
    for index in range(0, min_len):
        if m1_notes[index]=='0' or m2_notes[index]=='0':
            rests += 1
        note_diff = compare_notes(m1_notes[index],m2_notes[index])
        measure_diff+=note_diff
    measure_diff /= (min_len-rests)
    measure_diff = round(measure_diff,2)
    return measure_diff
        

In [13]:
def checkeq_measures(m1,m2):
    if m1.getSos()==m2.getSos():
        return True
    else:
        return False

In [14]:
checkeq_measures(measures[1],measures[2])

False

In [15]:
compare_measures(measures[1],measures[2])

-0.86

In [16]:
compare_measures(measures[2],measures[3])

3.43

In [17]:
compare_measures(measures[2],measures[4])

2.86

# 4.2 Similarity

In [18]:
def similarityratio_measures(m1,m2):
    m1_notes = m1.getSos()
    m2_notes = m2.getSos()
    min_len = min(len(m1_notes),len(m2_notes))
    max_len = max(len(m1_notes),len(m2_notes))
    same_notes = 0
    for index in range(0, min_len):
        if m1_notes[index]==m2_notes[index]:
            same_notes += 1
    ratio = round(same_notes/max_len,2)
    return ratio
            

In [19]:
def get_similarity_map(measures,threshold):
    '''get similarity map given threshold for measures'''
    meas_sim_dict = {}
    for index1 in range(0,len(measures)):
        meas_sim_dict[index1]=[]
        for index2 in range(0,len(measures)):
            if index2!=index1:
                sim = similarityratio_measures(measures[index1],measures[index2])
                if sim>threshold:
                    meas_sim_dict[index1].append(index2)
    return meas_sim_dict


In [20]:
simMap =  get_similarity_map(measures,threshold=0.75)
simMap


{0: [],
 1: [9],
 2: [10],
 3: [],
 4: [12],
 5: [13],
 6: [14],
 7: [],
 8: [],
 9: [1],
 10: [2],
 11: [],
 12: [4],
 13: [5],
 14: [6],
 15: [],
 16: [],
 17: [21],
 18: [22],
 19: [23],
 20: [],
 21: [17],
 22: [18],
 23: [19],
 24: []}

In [21]:
similarityratio_measures(measures[2],measures[4])

0.12

In [22]:
similarityratio_measures(measures[2],measures[9])

0.14

# 5. GA

# 5.1 Note mutation

In [9]:
import pandas as pd

coocc_df = pd.read_pickle('note_coocc.pickle')

In [13]:
ls = coocc_df['A4']
ls = ls[ls>0]
ls = ls.sort_values(ascending=False)
ls

level_0
A4     5912.0
G4     4399.0
F4     4379.0
B4     3998.0
F#4    3671.0
        ...  
F1        1.0
B6        1.0
B7        1.0
Eb7       1.0
Fb5       1.0
Name: A4, Length: 86, dtype: float64

In [15]:
ls.index[1]

'G4'

In [24]:
import random

candidateAmt = 10

def mutateNote(note):
    '''mutates a note with probability proportional to noteoccurence'''
    #DEBUG
    #print('getting new note')
    if note.getSo() != '0': #ignore rests
        #get candidates
        candidates = coocc_df['A4']
        candidates = candidates[candidates>0]
        candidates = candidates.sort_values(ascending=False)
        #get new note
        newnote_index = random.randint(0,candidateAmt)
        new_note_so = candidates.index[newnote_index]
        #change note
        #DEBUG
        #print('changing note')
        if len(new_note_so)>2:
            note.step = new_note_so[0]
            note.octave = int(new_note_so[2])
            note.accidental = new_note_so[1]
        else:
            note.step = new_note_so[0]
            note.octave = int(new_note_so[1])
        #change accidentla
        note.accidental = None
        return True
    else:
        return False
    

In [30]:
ls.index[7]

'A6'

In [31]:
n = Note('quarter','A','4',None,None,False)
n.getSo()

'A4'

In [32]:
len(n.getSo())>2

False

In [33]:
mutateNote(n)
n.getSo()

'A4'

In [32]:
measMutProb = 0.9
def mutateMeasure(index,curr_measure):
    '''mutates a measure with prob measMutProb'''
    meas_sos = curr_measure.getSos()
    meas_len = len(meas_sos)
    mut_prob  = random.randint(0,100)/100
    mut_succ = False
    if mut_prob <= measMutProb:
        while not mut_succ:
            #DEBUG
            #print("calling mutating note...")
            mut_index = random.randint(0,meas_len-1)
            note_to_mut = curr_measure.notes[mut_index]
            prev_note_so = note_to_mut.getSo()
            mut_succ = mutateNote(note_to_mut)
            new_note_so = note_to_mut.getSo()

        #change similar notes in similar measures
        simMeasInd = simMap[index]
        for i in simMeasInd:
            iNotes = measures[i].notes
            for iNote in iNotes:
                if iNote.getSo() == prev_note_so:
                    if len(new_note_so)>2:
                        iNote.step = new_note_so[0]
                        iNote.octave = int(new_note_so[2])
                        iNote.accidental = new_note_so[1]
                    else:
                        iNote.step = new_note_so[0]
                        iNote.octave = int(new_note_so[1])
                    #change accidentla
                    iNote.accidental = None


In [38]:
measures[1].getSos()

['F4', 'A4', 'D5', 'D5', 'D5', 'A4', 'A4']

In [39]:
mutateMeasure(1,measures[1])
measures[1].getSos()

['F4', 'A4', 'D5', 'D5', 'D5', 'A4', 'A4']

# 6. GA as a class

In [82]:
import random
from mxlParsing import *
import math
import pandas as pd
import gc

class GAComposition:
    measSimThresh = 0
    cooc_df = None
    simMap = None
    measMutProb = 0
    scorePop = []
    popMax = 0
    scoreOriginal = None
    #nondefinable
    mutPerScoreThresh = 3
    scorePool = []
    
    def __init__(self, score = None, accomp_measures=None, measSimThresh = 0.75, coocPath = 'note_coocc.pickle', 
                 measMutProb = 0.1, recombProb = 0.1, popMax = 30):
        self.scoreOriginal = score
        self.cooc_df = pd.read_pickle(coocPath)
        self.simMap = get_similarity_map(measures,measSimThresh)
        self.measSimThresh = measSimThresh
        self.measMutProb = measMutProb
        self.recombProb = recombProb
        self.popMax = popMax
        #functions
        initPopulation()
        
    def initPopulation():
        for i in range (0, self.popMax):
            s = self.score.copyScore()
            mutateScore(s)
            self.scorePop.append(s)
    
    def mutateScore(score):
        #how many mutations
        scoreMutAmt = random.randint(1,self.mutPerScoreThresh)
        #where
        for mut in range(1,scoreMutAmt):
            index = random.randint(0,score.length-1)
            meas = score.measures[index]
            mutateMeasure(index,meas)
        
    def mutateGeneration():
        self.scorePool = []
        #copy the parents
        for s in self.scorePop:
            self.scorePool.append(s.copyScore())
        #mutate the offspring
        for s in self.scorePool:
            mutateScore(s)
        #add parents to pool
        for s in self.scorePop:
            self.scorePool.append(s.copyScore())
            
    def survivalSelection():
        self.scorePop = []
        score_df = pd.DataFrame()
        for index in self.scorePool:
            score_df = score_df.append(get_score_fitness(self.scorePool[index]))
        score_df.rest_index()
        score_rank_df = score_df.sort_values(by=0)
        for i in range(0,self.maxPop):
            good_score_idx = score_rank_df[i]
            self.scorePop.append(scorePool[good_score_idx])
        self.scorePool = []
        gc.collect()
    
    def evolve(generations=100):
        for i in range(0,generations):
            mutateGeneration()
            survivalSelection()
        
    def returnTop3():
        score_df = pd.DataFrame()
        top3 = []
        for index in self.scorePop:
            score_df = score_df.append(get_score_fitness(self.scorePop[index]))
        score_df.rest_index()
        score_rank_df = score_df.sort_values(by=0)
        for i in range(0,3):
            good_score_idx = score_rank_df[i]
            top3.append(scorePop[good_score_idx])
        return top3

In [48]:
mutPerScoreThresh = 3
def mutateScore(score):
    #how many mutations
    scoreMutAmt = random.randint(1,mutPerScoreThresh)
    #where
    for mut in range(1,scoreMutAmt):
        index = random.randint(0,score.length-1)
        meas = score.measures[index]
        mutateMeasure(index,meas)

In [49]:
score = Score(measures)

In [50]:
get_score_fitness(score)

2129.78

In [51]:
score2 = score.copyScore()

In [52]:
for i in range(0,100):
    mutateScore(score2)
get_score_fitness(score2)

1853.08

In [70]:
df = pd.DataFrame()


In [71]:
df = df.append([300])
df

,0
0,300


In [72]:
df = df.append([400,100,500,12000,600,200,900]).reset_index()
df

,index,0
0,0,300
1,0,400
2,1,100
3,2,500
4,3,12000
5,4,600
6,5,200
7,6,900


In [74]:
sdf = df.sort_values(by=0)
sdf

,index,0
2,1,100
6,5,200
0,0,300
1,0,400
3,2,500
5,4,600
7,6,900
4,3,12000


In [80]:
sdf.index[0]

2

In [29]:
import numpy as np
import random

tuples = []

while len(tuples)<42:

    newcoord = False
    t = (0,0)

    while not newcoord:
        x_c = random.randint(1,7)
        y_c = random.randint(1,6)
        t = (x_c,y_c)
        if t not in tuples:
            tuples.append(t)
            newcoord = True
            break

    #print(t)

In [30]:
len(tuples)

42

In [31]:
for i in tuples:
    print(i)

(2, 6)
(5, 6)
(7, 1)
(5, 1)
(4, 1)
(7, 2)
(4, 4)
(2, 1)
(2, 4)
(5, 5)
(6, 3)
(7, 3)
(2, 3)
(4, 5)
(3, 4)
(4, 3)
(3, 2)
(7, 6)
(2, 2)
(6, 4)
(1, 5)
(5, 2)
(1, 4)
(6, 2)
(3, 3)
(5, 3)
(3, 6)
(4, 2)
(4, 6)
(7, 5)
(6, 6)
(1, 6)
(1, 1)
(3, 1)
(5, 4)
(1, 2)
(6, 1)
(1, 3)
(7, 4)
(6, 5)
(3, 5)
(2, 5)
